In [1]:
# -- Pre-run: change to parent directory for importing library -- only run this once!
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
os.chdir(module_path)

In [2]:
# -- Notebook start
import folium

from datakit import Itinerum
import datakit_config

In [3]:
# initialize Itinerum library with config
itinerum = Itinerum(config=datakit_config)

# read .csvs defined in config if not already available
itinerum.setup(force=False)

In [4]:
# load all users and find a real one with many coordinates recorded
users = itinerum.load_all_users()
print(users[3].coordinates.count())
test_user = users[3]

INFO:datakit.main:Loading user from database: 1/19...
INFO:datakit.main:Loading user from database: 2/19...
INFO:datakit.main:Loading user from database: 3/19...
INFO:datakit.main:Loading user from database: 4/19...
INFO:datakit.main:Loading user from database: 5/19...
INFO:datakit.main:Loading user from database: 6/19...
INFO:datakit.main:Loading user from database: 7/19...
INFO:datakit.main:Loading user from database: 8/19...
INFO:datakit.main:Loading user from database: 9/19...
INFO:datakit.main:Loading user from database: 10/19...
INFO:datakit.main:Loading user from database: 11/19...
INFO:datakit.main:Loading user from database: 12/19...
INFO:datakit.main:Loading user from database: 13/19...
INFO:datakit.main:Loading user from database: 14/19...
INFO:datakit.main:Loading user from database: 15/19...
INFO:datakit.main:Loading user from database: 16/19...
INFO:datakit.main:Loading user from database: 17/19...
INFO:datakit.main:Loading user from database: 18/19...
INFO:datakit.main:L

122


In [5]:
# run trip detection on user with config parameters
parameters = {
    'subway_stations': itinerum.database.load_subway_entrances(),
    'break_interval_seconds': datakit_config.TRIP_DETECTION_BREAK_INTERVAL_SECONDS,
    'subway_buffer_meters': datakit_config.TRIP_DETECTION_SUBWAY_BUFFER_METERS,
    'cold_start_distance': datakit_config.TRIP_DETECTION_COLD_START_DISTANCE_METERS,
    'accuracy_cutoff_meters': datakit_config.TRIP_DETECTION_ACCURACY_CUTOFF_METERS    
}
results = itinerum.run_process(itinerum.process.trip_detection.triplab.algorithm, [test_user], parameters)

INFO:datakit.main:Processing user (26e2ebfe-3154-486d-a3e8-fc05f24af961) trips: 1/1...


In [6]:
# save detected trips for user
trips, summaries = results[test_user]
trip_points = []
for trip in trips.values():
    for point in trip:
        point['uuid'] = test_user.uuid
        trip_points.append(point)
itinerum.database.save_trips(trip_points)

# reload user to build trip objects (this should be done automatically by trip detection,
# perhaps there should be a wrapper in the .trip_detection module to perform this on all add algorithms)
test_user = itinerum.database.load_user(uuid=test_user.uuid)

# examine the user's trips
test_user.trips

{1: <Trip num=1 code=2>,
 2: <Trip num=2 code=202>,
 3: <Trip num=3 code=202>,
 4: <Trip num=4 code=202>,
 5: <Trip num=5 code=202>,
 6: <Trip num=6 code=1>,
 7: <Trip num=7 code=1>,
 8: <Trip num=8 code=202>,
 9: <Trip num=9 code=1>,
 10: <Trip num=10 code=102>,
 11: <Trip num=11 code=2>,
 12: <Trip num=12 code=202>,
 13: <Trip num=13 code=202>,
 14: <Trip num=14 code=202>,
 15: <Trip num=15 code=1>,
 16: <Trip num=16 code=1>,
 17: <Trip num=17 code=1>,
 18: <Trip num=18 code=201>}

In [9]:
# examine a trip's attributes
test_trip = test_user.trips[1]
print('distance:', test_trip.distance)
print('start:', test_trip.start_UTC, 'end:', test_trip.end_UTC)
print('code:', test_trip.trip_code)

distance: 2742.598006423388
start: 2018-01-31 16:01:10 end: 2018-01-31 16:14:18
code: 2


In [10]:
# plot trip on map
m = folium.Map()
coordinates = [(p.latitude, p.longitude) for p in test_trip.points]
polyline = folium.PolyLine(locations=coordinates).add_to(m)
m.fit_bounds(coordinates)
m